In [1]:
from os import listdir
from PIL import Image
from numpy import asarray
from matplotlib import pyplot
from os.path import isdir
import numpy as np

In [2]:
def extract_images(filename, required_size=(200, 200)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    image = image.resize(required_size)
    pixels = asarray(image, dtype=np.float32)
    return pixels

In [3]:
def load_images(directory):
    images = list()
    # enumerate files
    for filename in listdir(directory):
        # path
        path = directory + filename
        # get face
        img = extract_images(path)
        # store
        images.append(img)
    return images


In [4]:
def load_dataset(directory):
    X, y = list(), list()
    # enumerate folders, on per class
    for subdir in ['Covid', 'PNEUMONIA']:
        # path
        path = directory + subdir + '/'
        # skip any files that might be in the dir
        if not isdir(path):
            continue
        # load all faces in the subdirectory
        faces = load_images(path)
        # create labels
        labels = [subdir for _ in range(len(faces))]
        # summarize progress
        print('>loaded %d examples for class: %s' % (len(faces), subdir))
        # store
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

In [5]:
trainX, trainy = load_dataset('xray_dataset_covid19/')

>loaded 94 examples for class: Covid
>loaded 100 examples for class: PNEUMONIA


In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
trainy = le.fit_transform(trainy)

In [7]:
trainX.shape

(194, 200, 200, 3)

In [8]:
from sklearn.model_selection import train_test_split
trainX, testX, trainy, testy = train_test_split(trainX, trainy, test_size=0.5)

In [9]:
trainX.shape,trainy.shape,testX.shape,testy.shape

((97, 200, 200, 3), (97,), (97, 200, 200, 3), (97,))

# Convolutional Network

Now that we've gotten our feet we with a simple DNN, let's try something more advanced. Although the process is the same, we'll be working with some additional features:
- Convolution, pooling, and dropout layers for building more complex models
- Visualizing training with TensorBoard
- Validation and test set evaluation for measuring generalizability
- Exporting with SavedModel to save training progress and deploy trained models

In [10]:
import time
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}


import tensorflow as tf
from tensorflow.python.ops import summary_ops_v2
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, optimizers, metrics

In [11]:
CNN_model = tf.keras.Sequential([
    layers.Reshape(
        target_shape=[200, 200, 3],
        input_shape=(200, 200,3)),
    layers.Conv2D(2, 5, padding='same', activation=tf.nn.relu),
    layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    layers.Conv2D(4, 5, padding='same', activation=tf.nn.relu),
    layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    layers.Flatten(),
    layers.Dense(32, activation=tf.nn.relu),
    layers.Dropout(rate=0.4),
    layers.Dense(3)])

optimizer = optimizers.SGD(learning_rate=0.01, momentum=0.5)

In [12]:
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [13]:
def train_step(model, optimizer, images, labels):

    # Record the operations used to compute the loss, so that the gradient
    # of the loss with respect to the variables can be computed.
    with tf.GradientTape() as tape:
        logits = model(images, training=True)
        loss = compute_loss(labels, logits)
        compute_accuracy(labels, logits)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [14]:
def train(model, optimizer, dataset, log_freq=50):
    """
    Trains model on `dataset` using `optimizer`.
    """
    # Metrics are stateful. They accumulate values and return a cumulative
    # result when you call .result(). Clear accumulated values with .reset_states()
    avg_loss = metrics.Mean('loss', dtype=tf.float32)

    # Datasets can be iterated over like any other Python iterable.
    for images, labels in dataset:
        loss = train_step(model, optimizer, images, labels)
        avg_loss(loss)

        if tf.equal(optimizer.iterations % log_freq, 0):
            # summary_ops_v2.scalar('loss', avg_loss.result(), step=optimizer.iterations)
            # summary_ops_v2.scalar('accuracy', compute_accuracy.result(), step=optimizer.iterations)
            print('step:', int(optimizer.iterations),
                  'loss:', avg_loss.result().numpy(),
                  'acc:', compute_accuracy.result().numpy())
            avg_loss.reset_states()
            compute_accuracy.reset_states()

In [15]:
def test(model, dataset, step_num):
    """
    Perform an evaluation of `model` on the examples from `dataset`.
    """
    avg_loss = metrics.Mean('loss', dtype=tf.float32)

    for (images, labels) in dataset:
        logits = model(images, training=False)
        avg_loss(compute_loss(labels, logits))
        compute_accuracy(labels, logits)

    print('Model test set loss: {:0.4f} accuracy: {:0.2f}%'.format(
        avg_loss.result(), compute_accuracy.result() * 100))

    print('loss:', avg_loss.result(), 'acc:', compute_accuracy.result())
    # summary_ops_v2.scalar('loss', avg_loss.result(), step=step_num)
    # summary_ops_v2.scalar('accuracy', compute_accuracy.result(), step=step_num)

In [16]:
NUM_TRAIN_EPOCHS = 200

for i in range(NUM_TRAIN_EPOCHS):
    start = time.time()
    train_ds = tf.data.Dataset.from_tensor_slices((trainX, trainy))
    train_ds = train_ds.shuffle(200).batch(100)
    #   with train_summary_writer.as_default():
    train(CNN_model, optimizer, train_ds, log_freq=500)
    end = time.time()
    print('Train time for epoch #{} ({} total steps): {}'.format(
        i + 1, int(optimizer.iterations), end - start))
    test_ds = tf.data.Dataset.from_tensor_slices((testX, testy))
    test_ds = test_ds.batch(80)
    test(CNN_model, test_ds, optimizer.iterations)

Train time for epoch #1 (1 total steps): 2.382455825805664
Model test set loss: 6.5183 accuracy: 10.31%
loss: tf.Tensor(6.5182695, shape=(), dtype=float32) acc: tf.Tensor(0.10309278, shape=(), dtype=float32)
Train time for epoch #2 (2 total steps): 2.4777748584747314
Model test set loss: 1.0361 accuracy: 27.58%
loss: tf.Tensor(1.0361073, shape=(), dtype=float32) acc: tf.Tensor(0.2757732, shape=(), dtype=float32)
Train time for epoch #3 (3 total steps): 2.3673200607299805
Model test set loss: 0.7117 accuracy: 35.40%
loss: tf.Tensor(0.7117114, shape=(), dtype=float32) acc: tf.Tensor(0.3539519, shape=(), dtype=float32)
Train time for epoch #4 (4 total steps): 2.3240468502044678
Model test set loss: 0.6975 accuracy: 38.53%
loss: tf.Tensor(0.69751537, shape=(), dtype=float32) acc: tf.Tensor(0.38530928, shape=(), dtype=float32)
Train time for epoch #5 (5 total steps): 2.2381420135498047
Model test set loss: 0.6971 accuracy: 40.52%
loss: tf.Tensor(0.6970622, shape=(), dtype=float32) acc: tf.T

Model test set loss: 0.6929 accuracy: 49.10%
loss: tf.Tensor(0.69294095, shape=(), dtype=float32) acc: tf.Tensor(0.49097937, shape=(), dtype=float32)
Train time for epoch #41 (41 total steps): 2.557220220565796
Model test set loss: 0.6929 accuracy: 49.15%
loss: tf.Tensor(0.6929306, shape=(), dtype=float32) acc: tf.Tensor(0.49145085, shape=(), dtype=float32)
Train time for epoch #42 (42 total steps): 2.578850269317627
Model test set loss: 0.6930 accuracy: 49.08%
loss: tf.Tensor(0.693042, shape=(), dtype=float32) acc: tf.Tensor(0.49079528, shape=(), dtype=float32)
Train time for epoch #43 (43 total steps): 3.3559648990631104
Model test set loss: 0.6931 accuracy: 49.14%
loss: tf.Tensor(0.6931287, shape=(), dtype=float32) acc: tf.Tensor(0.49136898, shape=(), dtype=float32)
Train time for epoch #44 (44 total steps): 3.602038860321045
Model test set loss: 0.6932 accuracy: 49.17%
loss: tf.Tensor(0.69324005, shape=(), dtype=float32) acc: tf.Tensor(0.4916823, shape=(), dtype=float32)
Train time

Model test set loss: 0.6926 accuracy: 50.15%
loss: tf.Tensor(0.6926228, shape=(), dtype=float32) acc: tf.Tensor(0.5015007, shape=(), dtype=float32)
Train time for epoch #80 (80 total steps): 2.4493470191955566
Model test set loss: 0.6925 accuracy: 50.13%
loss: tf.Tensor(0.69254863, shape=(), dtype=float32) acc: tf.Tensor(0.50128865, shape=(), dtype=float32)
Train time for epoch #81 (81 total steps): 2.4646120071411133
Model test set loss: 0.6925 accuracy: 50.17%
loss: tf.Tensor(0.6925075, shape=(), dtype=float32) acc: tf.Tensor(0.50165457, shape=(), dtype=float32)
Train time for epoch #82 (82 total steps): 2.3916893005371094
Model test set loss: 0.6925 accuracy: 50.14%
loss: tf.Tensor(0.692479, shape=(), dtype=float32) acc: tf.Tensor(0.50138295, shape=(), dtype=float32)
Train time for epoch #83 (83 total steps): 2.4235281944274902
Model test set loss: 0.6925 accuracy: 50.14%
loss: tf.Tensor(0.69248295, shape=(), dtype=float32) acc: tf.Tensor(0.50142837, shape=(), dtype=float32)
Train t

Model test set loss: 0.6924 accuracy: 50.31%
loss: tf.Tensor(0.6923674, shape=(), dtype=float32) acc: tf.Tensor(0.5031452, shape=(), dtype=float32)
Train time for epoch #119 (119 total steps): 2.415929079055786
Model test set loss: 0.6924 accuracy: 50.32%
loss: tf.Tensor(0.69236916, shape=(), dtype=float32) acc: tf.Tensor(0.5031621, shape=(), dtype=float32)
Train time for epoch #120 (120 total steps): 2.40049409866333
Model test set loss: 0.6924 accuracy: 50.31%
loss: tf.Tensor(0.69236517, shape=(), dtype=float32) acc: tf.Tensor(0.50309277, shape=(), dtype=float32)
Train time for epoch #121 (121 total steps): 2.41888165473938
Model test set loss: 0.6924 accuracy: 50.26%
loss: tf.Tensor(0.6923533, shape=(), dtype=float32) acc: tf.Tensor(0.5026412, shape=(), dtype=float32)
Train time for epoch #122 (122 total steps): 2.367280960083008
Model test set loss: 0.6924 accuracy: 50.26%
loss: tf.Tensor(0.69235325, shape=(), dtype=float32) acc: tf.Tensor(0.50261956, shape=(), dtype=float32)
Train

Train time for epoch #157 (157 total steps): 2.439507007598877
Model test set loss: 0.6928 accuracy: 50.39%
loss: tf.Tensor(0.69277215, shape=(), dtype=float32) acc: tf.Tensor(0.50393987, shape=(), dtype=float32)
Train time for epoch #158 (158 total steps): 2.4086358547210693
Model test set loss: 0.6928 accuracy: 50.41%
loss: tf.Tensor(0.692843, shape=(), dtype=float32) acc: tf.Tensor(0.5041433, shape=(), dtype=float32)
Train time for epoch #159 (159 total steps): 2.357584238052368
Model test set loss: 0.6926 accuracy: 50.41%
loss: tf.Tensor(0.69261, shape=(), dtype=float32) acc: tf.Tensor(0.50411725, shape=(), dtype=float32)
Train time for epoch #160 (160 total steps): 2.4057531356811523
Model test set loss: 0.6926 accuracy: 50.43%
loss: tf.Tensor(0.69258934, shape=(), dtype=float32) acc: tf.Tensor(0.5042848, shape=(), dtype=float32)
Train time for epoch #161 (161 total steps): 2.368582248687744
Model test set loss: 0.6925 accuracy: 50.42%
loss: tf.Tensor(0.6924649, shape=(), dtype=fl

Train time for epoch #196 (196 total steps): 2.3607659339904785
Model test set loss: 0.6927 accuracy: 50.53%
loss: tf.Tensor(0.6927086, shape=(), dtype=float32) acc: tf.Tensor(0.50528616, shape=(), dtype=float32)
Train time for epoch #197 (197 total steps): 2.4353790283203125
Model test set loss: 0.6925 accuracy: 50.54%
loss: tf.Tensor(0.69245446, shape=(), dtype=float32) acc: tf.Tensor(0.5054163, shape=(), dtype=float32)
Train time for epoch #198 (198 total steps): 2.417189836502075
Model test set loss: 0.6924 accuracy: 50.53%
loss: tf.Tensor(0.6923752, shape=(), dtype=float32) acc: tf.Tensor(0.5052848, shape=(), dtype=float32)
Train time for epoch #199 (199 total steps): 2.409574270248413
Model test set loss: 0.6924 accuracy: 50.54%
loss: tf.Tensor(0.69238615, shape=(), dtype=float32) acc: tf.Tensor(0.5053878, shape=(), dtype=float32)
Train time for epoch #200 (200 total steps): 2.363121747970581
Model test set loss: 0.6924 accuracy: 50.54%
loss: tf.Tensor(0.6923847, shape=(), dtype=

In [17]:
CNN_model.save('CNN_model_covid_pneumonia')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: CNN_model_covid_pneumonia/assets


# ResNet 

Here we train a tf.keras implementation of ResNet-18

Includes cell dividers for running with IPython!

![](resnet.jpeg)

In [18]:
def conv3x3(channels, stride=1, kernel=(3, 3)):
    return keras.layers.Conv2D(channels, kernel, strides=stride, padding='same',
                               use_bias=False,
                            kernel_initializer=tf.random_normal_initializer())

In [19]:
class ResnetBlock(keras.Model):

    def __init__(self, channels, strides=1, residual_path=False):
        super(ResnetBlock, self).__init__()

        self.channels = channels
        self.strides = strides
        self.residual_path = residual_path

        self.conv1 = conv3x3(channels, strides)
        self.bn1 = keras.layers.BatchNormalization()
        self.conv2 = conv3x3(channels)
        self.bn2 = keras.layers.BatchNormalization()

        if residual_path:
            self.down_conv = conv3x3(channels, strides, kernel=(1, 1))
            self.down_bn = tf.keras.layers.BatchNormalization()

    def call(self, inputs, training=None):
        residual = inputs

        x = self.bn1(inputs, training=training)
        x = tf.nn.relu(x)
        x = self.conv1(x)
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)

        # this module can be added into self.
        # however, module in for can not be added.
        if self.residual_path:
            residual = self.down_bn(inputs, training=training)
            residual = tf.nn.relu(residual)
            residual = self.down_conv(residual)

        x = x + residual
        return x

In [20]:
class ResNet(keras.Model):

    def __init__(self, block_list, num_classes, initial_filters=16, **kwargs):
        super(ResNet, self).__init__(**kwargs)

        self.num_blocks = len(block_list)
        self.block_list = block_list

        self.in_channels = initial_filters
        self.out_channels = initial_filters
        self.conv_initial = conv3x3(self.out_channels)

        self.blocks = keras.models.Sequential(name='dynamic-blocks')

        # build all the blocks
        for block_id in range(len(block_list)):
            for layer_id in range(block_list[block_id]):

                if block_id != 0 and layer_id == 0:
                    block = ResnetBlock(self.out_channels, strides=2, residual_path=True)
                else:
                    if self.in_channels != self.out_channels:
                        residual_path = True
                    else:
                        residual_path = False
                    block = ResnetBlock(self.out_channels, residual_path=residual_path)

                self.in_channels = self.out_channels

                self.blocks.add(block)

            self.out_channels *= 2

        self.final_bn = keras.layers.BatchNormalization()
        self.avg_pool = keras.layers.GlobalAveragePooling2D()
        self.fc = keras.layers.Dense(num_classes)

    def call(self, inputs, training=None):

        out = self.conv_initial(inputs)

        out = self.blocks(out, training=training)

        out = self.final_bn(out, training=training)
        out = tf.nn.relu(out)

        out = self.avg_pool(out)
        out = self.fc(out)


        return out

In [22]:
num_classes = 3
batch_size = 32
epochs = 200

# build model and optimizer
ResNet_model = ResNet([16, 8, 4], num_classes)
ResNet_model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
ResNet_model.build(input_shape=(None, 200,200,3))
print("Number of variables in the model :", len(ResNet_model.variables))
ResNet_model.summary()

y_ohe = tf.one_hot(trainy, depth=3).numpy()
testy_ohe = tf.one_hot(testy, depth=3).numpy()

# train
ResNet_model.fit(trainX, y_ohe, batch_size=batch_size, epochs=epochs,
          validation_data=(testX, testy_ohe), verbose=1)

# evaluate on test set
scores = ResNet_model.evaluate(testX, testy_ohe, batch_size, verbose=1)
print("Final test loss and accuracy :", scores)

Number of variables in the model : 297
Model: "res_net_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_85 (Conv2D)           multiple                  432       
_________________________________________________________________
dynamic-blocks (Sequential)  multiple                  501760    
_________________________________________________________________
batch_normalization_145 (Bat multiple                  256       
_________________________________________________________________
global_average_pooling2d_1 ( multiple                  0         
_________________________________________________________________
dense_5 (Dense)              multiple                  195       
Total params: 502,643
Trainable params: 499,443
Non-trainable params: 3,200
_________________________________________________________________
Train on 97 samples, validate on 97 samples
Epoch 1/200
97/97 [===========

KeyboardInterrupt: 

In [23]:
tf.saved_model.save(ResNet_model,'ResNet_Model_Covid_Pneumonia')

INFO:tensorflow:Assets written to: ResNet_Model_Covid_Pneumonia/assets


# VGG16

Fitting a VGG-16 network for image classification!
We use gradient clipping for faster convergence.

A complete implementation of VGG-16 is available in network.py

![](vgg16.png)

In [11]:
from tensorflow.keras import regularizers

In [12]:
class VGG16(models.Model):


    def __init__(self, input_shape, num_classes):
        """

        :param input_shape: [32, 32, 3]
        """
        super(VGG16, self).__init__()

        weight_decay = 0.000
        self.num_classes = num_classes

        model = models.Sequential()

        model.add(layers.Conv2D(64, (3, 3), padding='same',
                         input_shape=input_shape, kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.3))

        model.add(layers.Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

        model.add(layers.Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

        model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))


        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))


        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Dropout(0.5))

        model.add(layers.Flatten())
        model.add(layers.Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(self.num_classes))
        # model.add(layers.Activation('softmax'))


        self.model = model


    def call(self, x):

        x = self.model(x)

        return x

In [13]:
def compute_loss(logits, labels):
    return tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits=logits, labels=labels))

In [16]:
VGG16_model = VGG16([200, 200, 3], 3)


# must specify from_logits=True!
criteon = keras.losses.CategoricalCrossentropy(from_logits=True)
metric = keras.metrics.CategoricalAccuracy()

optimizer = optimizers.Adam(learning_rate=0.0001)

db_train = tf.data.Dataset.from_tensor_slices((trainX, trainy)).batch(256)
db_test = tf.data.Dataset.from_tensor_slices((testX, testy)).batch(256)

for epoch in range(200):

    for step, (x, y) in enumerate(db_train):
        # [b, 1] => [b]
        # y = tf.squeeze(y, axis=1)
        # [b, 10]
        y = tf.one_hot(y, depth=3)

        with tf.GradientTape() as tape:
            logits = VGG16_model(x)
            loss = criteon(y, logits)
            # loss2 = compute_loss(logits, tf.argmax(y, axis=1))
            # mse_loss = tf.reduce_sum(tf.square(y-logits))
            # print(y.shape, logits.shape)
            metric.update_state(y, logits)

        grads = tape.gradient(loss, VGG16_model.trainable_variables)
        # MUST clip gradient here or it will disconverge!
        grads = [ tf.clip_by_norm(g, 15) for g in grads]
        optimizer.apply_gradients(zip(grads, VGG16_model.trainable_variables))

        if step % 40 == 0:
            # for g in grads:
            #     print(tf.norm(g).numpy())
            print(epoch, step, 'loss:', float(loss), 'acc:', metric.result().numpy())
            metric.reset_states()


    if epoch % 1 == 0:

        metric = keras.metrics.CategoricalAccuracy()
        for x, y in db_test:
            # [b, 1] => [b]
            # y = tf.squeeze(y, axis=1)
            # [b, 10]
            y = tf.one_hot(y, depth=3)

            logits = VGG16_model.predict(x)
            # be careful, these functions can accept y as [b] without warnning.
            metric.update_state(y, logits)
        print('test acc:', metric.result().numpy())
        metric.reset_states()

0 0 loss: 1.0220255851745605 acc: 0.5154639
test acc: 0.48453608
1 0 loss: 1.2034764289855957 acc: 0.48453608
test acc: 0.5154639
2 0 loss: 2.202988624572754 acc: 0.5154639
test acc: 0.52577317
3 0 loss: 0.645288348197937 acc: 0.556701
test acc: 0.48453608
4 0 loss: 0.7542040944099426 acc: 0.48453608
test acc: 0.48453608
5 0 loss: 0.6500425338745117 acc: 0.49484536
test acc: 0.5154639
6 0 loss: 0.634027361869812 acc: 0.556701
test acc: 0.62886596
7 0 loss: 0.5975534915924072 acc: 0.68041235
test acc: 0.9484536
8 0 loss: 0.5367076396942139 acc: 0.96907216
test acc: 0.88659793
9 0 loss: 0.4725761413574219 acc: 0.9484536
test acc: 0.9072165
10 0 loss: 0.37413936853408813 acc: 0.9587629
test acc: 0.9484536
11 0 loss: 0.25634706020355225 acc: 0.97938144
test acc: 0.8969072
12 0 loss: 0.20906873047351837 acc: 0.9587629
test acc: 0.6082474
13 0 loss: 0.6709789037704468 acc: 0.6494845
test acc: 0.9587629
14 0 loss: 0.0733010321855545 acc: 0.96907216
test acc: 0.8453608
15 0 loss: 0.33787265419

KeyboardInterrupt: 

In [17]:
tf.saved_model.save(VGG16_model,'VGG16_Model_Covid_Pneumonia')

INFO:tensorflow:Assets written to: VGG16_Model_Covid_Pneumonia/assets
